In [18]:
import cv2
import numpy as np

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from glob import glob

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.applications import MobileNetV2

In [20]:
global image_h
global image_w
global num_landmarks

""" Hyperparameters """
image_h = 512
image_w = 512
num_landmarks = 106
input_shape = (image_h, image_w, 3)
batch_size = 32
lr = 1e-3
num_epochs = 100

In [21]:
""" Paths """
path = "./LaPa"

In [22]:
train_img = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
train_land = sorted(glob(os.path.join(path, "train", "landmarks", "*.txt")))
valid_img = sorted(glob(os.path.join(path, "val", "images", "*.jpg")))
valid_land = sorted(glob(os.path.join(path, "val", "landmarks", "*.txt")))
test_img = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
test_land = sorted(glob(os.path.join(path, "test", "landmarks", "*.txt")))
train_img

['./LaPa\\train\\images\\10001436895_0.jpg',
 './LaPa\\train\\images\\10003645676_0.jpg',
 './LaPa\\train\\images\\10004446093_1.jpg',
 './LaPa\\train\\images\\10004446093_17.jpg',
 './LaPa\\train\\images\\10004446093_2.jpg',
 './LaPa\\train\\images\\10004446093_3.jpg',
 './LaPa\\train\\images\\10004446093_4.jpg',
 './LaPa\\train\\images\\10004446093_7.jpg',
 './LaPa\\train\\images\\10004446093_8.jpg',
 './LaPa\\train\\images\\10006787746_6.jpg',
 './LaPa\\train\\images\\10009875486_0.jpg',
 './LaPa\\train\\images\\10009875486_1.jpg',
 './LaPa\\train\\images\\10011648266_14.jpg',
 './LaPa\\train\\images\\100121306_0.jpg',
 './LaPa\\train\\images\\10012551673_0.jpg',
 './LaPa\\train\\images\\10012551673_1.jpg',
 './LaPa\\train\\images\\10012551673_10.jpg',
 './LaPa\\train\\images\\10012551673_11.jpg',
 './LaPa\\train\\images\\10012551673_13.jpg',
 './LaPa\\train\\images\\10012551673_6.jpg',
 './LaPa\\train\\images\\10012551673_7.jpg',
 './LaPa\\train\\images\\10012551673_9.jpg',
 './LaP

In [23]:
#train_img = train_img.decode()

In [24]:
def read_image_lankmarks(image_path, landmark_path):
    """ Image """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    h, w, _ = image.shape
    image = cv2.resize(image, (image_w, image_h))
    
    # Cast the image data to float32
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.0
  #  image = image.astype(np.float32)

    """ Lankmarks """
    data = open(landmark_path, "r").read()
    lankmarks = []

    for line in data.strip().split("\n")[1:]:
        x, y = line.split(" ")
        x = float(x)/w
        y = float(y)/h

        lankmarks.append(x)
        lankmarks.append(y)

    lankmarks = np.array(lankmarks, dtype=np.float32)

    return image, lankmarks

In [25]:
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image, landmarks = read_image_lankmarks(x, y)
        return image, landmarks

    image, landmarks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([image_h, image_w, 3])
    landmarks.set_shape([num_landmarks * 2])

    return image, landmarks

In [26]:
def build_model(input_shape, num_landmarks):
    inputs = L.Input(input_shape)

    backbone = MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs, alpha=0.5)
    backbone.trainable = True

    x = backbone.output
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.2)(x)
    outputs = L.Dense(num_landmarks*2, activation="sigmoid")(x)

    model = tf.keras.models.Model(inputs, outputs)
    return model

In [27]:
ds_train = tf.data.Dataset.from_tensor_slices((train_img, train_land))
ds_valid = tf.data.Dataset.from_tensor_slices((valid_img, valid_land))

for i in ds_train:
    print(i)
    print(i[0])
    print(i[1])
    print('-----------------------------------------------------------------------------------------') 

(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\10001436895_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\10001436895_0.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\10001436895_0.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\10001436895_0.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\10003645676_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\10003645676_0.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\10003645676_0.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\10003645676_0.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\10004446093_1.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaP

(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\12083782064_3.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\12083782064_3.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\12083782064_3.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\12083782064_3.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\12089390433_11.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\12089390433_11.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\12089390433_11.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\12089390433_11.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\12089390433_12.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'

tf.Tensor(b'./LaPa\\train\\landmarks\\189274833_7.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\189439776_1.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\189439776_1.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\189439776_1.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\189439776_1.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\189839728_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\189839728_0.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\189839728_0.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\189839728_0.txt', shape=(), dtype=string)
--------------------------------------------------------------------------

(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\2740737595_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\2740737595_0.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\2740737595_0.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\2740737595_0.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\2740839966_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\2740839966_0.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\2740839966_0.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\2740839966_0.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\2741264852_0.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\

(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\3588393627_2.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\3588393627_2.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\3588393627_2.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\3588393627_2.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\3588393627_3.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\landmarks\\3588393627_3.txt'>)
tf.Tensor(b'./LaPa\\train\\images\\3588393627_3.jpg', shape=(), dtype=string)
tf.Tensor(b'./LaPa\\train\\landmarks\\3588393627_3.txt', shape=(), dtype=string)
-----------------------------------------------------------------------------------------
(<tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\\images\\3588393627_4.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'./LaPa\\train\

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
ds_train = ds_train.shuffle(buffer_size=5000)
ds_valid = ds_valid.shuffle(buffer_size=5000)
ds_train

<_ShuffleDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [29]:
ds_train = ds_train.map(preprocess)
ds_valid = ds_valid.map(preprocess)
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(212,), dtype=tf.float32, name=None))>

In [30]:
batch=8
ds_train = ds_train.batch(batch).prefetch(2)
ds_valid = ds_valid.batch(batch).prefetch(2)
# Most dataset input pipelines should end with a call to prefetch. 
# This allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
ds_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 212), dtype=tf.float32, name=None))>

In [31]:
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

In [32]:
""" Model """
model = build_model(input_shape, num_landmarks)
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

""" Training """
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

In [33]:
 """ Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model.fit(ds_train,
        validation_data=ds_train,
        epochs=num_epochs,
        callbacks=callbacks
    )

In [ ]:
# ds_train = ds_train.decode()
# # https://velog.io/@mmzgpgp/AttributeError-str-object-has-no-attribute-decode-%EC%97%90%EB%9F%AC

# # 파이썬에서 AttributeError: 'str' object has no attribute 'decode' 이런 메시지가 뜨시면
# # 해당 코드 줄의 .decode() 부분을 지우시면 됩니다.
# # 이미 decode가 되었기 때문에 decode할게 없다는 뜻 입니다.